In [18]:
import numpy as np

from dipy.core.gradients import gradient_table
from dipy.data import get_fnames, default_sphere
from dipy.direction import ProbabilisticDirectionGetter
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, load_nifti_data
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_trk, load_trk
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.tracking.local_tracking import (LocalTracking,
                                          ParticleFilteringTracking)
from dipy.tracking.streamline import Streamlines
from dipy.tracking import utils
from dipy.viz import window, actor, colormap, has_fury

In [2]:
# Enables/disables interactive visualization
interactive = False

In [ ]:
hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')
label_fname = get_fnames('stanford_labels')
f_pve_csf, f_pve_gm, f_pve_wm = get_fnames('stanford_pve_maps')

In [3]:
data, affine, hardi_img = load_nifti(hardi_fname, return_img=True)
labels = load_nifti_data(label_fname)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

pve_csf_data = load_nifti_data(f_pve_csf)
pve_gm_data = load_nifti_data(f_pve_gm)
pve_wm_data, _, voxel_size = load_nifti(f_pve_wm, return_voxsize=True)

shape = labels.shape

response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.7)
csd_model = ConstrainedSphericalDeconvModel(gtab, response)
csd_fit = csd_model.fit(data, mask=pve_wm_data)

dg = ProbabilisticDirectionGetter.from_shcoeff(csd_fit.shm_coeff,
                                               max_angle=20.,
                                               sphere=default_sphere)

In [4]:
seed_mask = (labels == 2)

In [5]:
np.sum(seed_mask)

505

In [6]:
seed_mask = (labels == 2) + (labels == 1)

In [7]:
np.sum(seed_mask)

58788

In [8]:
seed_mask[pve_wm_data < 0.5] = 0
seeds = utils.seeds_from_mask(seed_mask, affine, density=2)

"""
CMC/ACT Stopping Criterion
==========================
Continuous map criterion (CMC) [Girard2014]_ and Anatomically-constrained
tractography (ACT) [Smith2012]_ both uses PVEs information from
anatomical images to determine when the tractography stops.
Both stopping criterion use a trilinear interpolation
at the tracking position. CMC stopping criterion uses a probability derived
from the PVE maps to determine if the streamline reaches a 'valid' or 'invalid'
region. ACT uses a fixed threshold on the PVE maps. Both stopping criterion can
be used in conjunction with PFT. In this example, we used CMC.
"""

from dipy.tracking.stopping_criterion import CmcStoppingCriterion

voxel_size = np.average(voxel_size[1:4])
step_size = 0.2

cmc_criterion = CmcStoppingCriterion.from_pve(pve_wm_data,
                                              pve_gm_data,
                                              pve_csf_data,
                                              step_size=step_size,
                                              average_voxel_size=voxel_size)

# Particle Filtering Tractography
pft_streamline_generator = ParticleFilteringTracking(dg,
                                                     cmc_criterion,
                                                     seeds,
                                                     affine,
                                                     max_cross=1,
                                                     step_size=step_size,
                                                     maxlen=1000,
                                                     pft_back_tracking_dist=2,
                                                     pft_front_tracking_dist=1,
                                                     particle_count=15,
                                                     return_all=False)
streamlines = Streamlines(pft_streamline_generator)

sft = StatefulTractogram(streamlines, hardi_img, Space.RASMM)
save_trk(sft, "tractogram_pft.trk")

# if has_fury:
#     r = window.Renderer()
#     r.add(actor.line(streamlines, colormap.line_colors(streamlines)))
#     window.record(r, out_path='tractogram_pft.png',
#                   size=(800, 800))
#     if interactive:
#         window.show(r)

# """
# .. figure:: tractogram_pft.png
#  :align: center

#  **Corpus Callosum using particle filtering tractography**
# """

# # Local Probabilistic Tractography
# prob_streamline_generator = LocalTracking(dg,
#                                           cmc_criterion,
#                                           seeds,
#                                           affine,
#                                           max_cross=1,
#                                           step_size=step_size,
#                                           maxlen=1000,
#                                           return_all=False)
# streamlines = Streamlines(prob_streamline_generator)
# sft = StatefulTractogram(streamlines, hardi_img, Space.RASMM)
# save_trk(sft, "tractogram_probabilistic_cmc.trk")

# if has_fury:
#     r = window.Renderer()
#     r.add(actor.line(streamlines, colormap.line_colors(streamlines)))
#     window.record(r, out_path='tractogram_probabilistic_cmc.png',
#                   size=(800, 800))
#     if interactive:
#         window.show(r)

In [20]:
sft = load_trk('./tractogram_pft.trk', hardi_img)

In [21]:
streamlines = sft.streamlines

In [22]:
from dipy.data import read_stanford_labels, fetch_stanford_t1, read_stanford_t1

hardi_img, gtab, labels_img = read_stanford_labels()
data = hardi_img.get_fdata()
labels = labels_img.get_fdata()

fetch_stanford_t1()
t1 = read_stanford_t1()
t1_data = t1.get_fdata()

In [58]:
from dipy.viz import window, actor, colormap as cmap

# Enables/disables interactive visualization
interactive = False

# Make display objects
color = cmap.line_colors(streamlines)
cc_streamlines_actor = actor.line(streamlines,
                                  cmap.line_colors(streamlines))

vol_actor = actor.slicer(t1_data, affine=t1.affine)

vol_actor.display(x=40)
vol_actor2 = vol_actor.copy()
vol_actor2.display(z=35)

# Add display objects to canvas
r = window.Renderer()
r.add(vol_actor)
r.add(vol_actor2)
r.add(cc_streamlines_actor)

r.set_camera(position=[-1, 0, 0.3], focal_point=[0, 0, 0], view_up=[0, 0, 1])

In [59]:
# window.show(r)

In [60]:
window.record(r, n_frames=360, out_path='./pft/fig', path_numbering=True, az_ang=1,
            size=(800, 800))

In [61]:
ll = glob('./pft/*.png')

In [62]:
ll.sort()

In [63]:
import moviepy.editor as mp
from glob import glob
imseq = mp.ImageSequenceClip(ll, fps=18)
imseq.write_videofile('./whole_brain.mp4')

t:   1%|▏         | 5/361 [00:00<00:08, 42.56it/s, now=None]

Moviepy - Building video ./whole_brain.mp4.
Moviepy - Writing video ./whole_brain.mp4



Moviepy - Done !
Moviepy - video ready ./whole_brain.mp4
